# Arm and Gripper

In the previous tutorial, we explored the concept of *goto* and basic control. Now, let's delve deeper into what we can do with Reachy's arms and grippers.

## Initialize Your Robot

First, connect to your robot:

In [ ]:
from reachy2_sdk import ReachySDK
import time

reachy = ReachySDK(host='localhost')  # Replace with the actual IP or the .local adress of your robot

Next, we need to turn on the parts we want to use:

In [2]:
reachy.l_arm.turn_on()
reachy.r_arm.turn_on()

Since the grippers are part of the arms, they will also be turned on. You could, of course, turn on the whole robot by calling `reachy.turn_on()` directly.
Let's check if the arms are on:

In [ ]:
print(reachy.r_arm.is_on())
print(reachy.l_arm.is_on())

## Control the arms

Arms can be controlled in two spaces:

* the **joint space**, which allows to read and write directly the angle values of each joint of the arm
* the **cartesian space**, which consists in controlling the end effector position and orientation in Reachy's coordinate system

> Both spaces are quite different, and **we advise not to mix them** if you are not familiar with the output.
In fact, values of the joint space are expressed in each actuator's coordinate system (respectively shoulder, elbow and wrist), whereas commands in cartesian space are expressed in Reachy's coordinate system

### Joint space

Reachy's arm offers 7 degrees of freedom. It also gives access to one joint for the gripper.
The arm is divided as follow:
- shoulder, composed of 2 joints (pitch and roll)
- elbow, composed of 2 joints (yaw and pitch)
- wrist, composed of 3 joints (roll, pitch and yaw)

We refer to the shoulder, elbow and wrist as actuators.
For some actions, such as changing the compliancy, it is the lowest level of control you will have.

You can inspect the details of the arm with:

In [ ]:
reachy.r_arm.joints

In [ ]:
reachy.l_arm.joints

You can easily access the position of each joint in one call with `get_joint_positions()`

In [ ]:
reachy.r_arm.get_joints_positions(round=3)

In [ ]:
reachy.l_arm.get_joints_positions(round=3)

#### Move the arms in joint space

The simplest way to move an arm is to set the angle of each joint. Define a joint positions list:

In [3]:
r_elbow_at_90_deg = [0, -15, -15, -90, 0, 0, 0]
l_elbow_at_90_deg = [0, 15, 15, -90, 0, 0, 0]

Send the joint goal positions to the arm with `goto_joints()`

In [ ]:
reachy.r_arm.goto_joints(r_elbow_at_90_deg)
reachy.l_arm.goto_joints(l_elbow_at_90_deg, wait = True)


In order to easily move a single joint, we can use the `goto()` method on a given joint:

In [ ]:
reachy.r_arm.elbow.pitch.goto(0)

This method works exactly like a goto on the part. All gotos sent to joints are stacked on the part they belong to.
For example:

In [ ]:
part_goto_id = reachy.r_arm.goto_joints([-10, -15, -15, -90, 0, 0, 0])
joint_goto_id = reachy.r_arm.elbow.pitch.goto(0)

print(part_goto_id)
print(joint_goto_id)

print(reachy.r_arm.get_goto_queue())

`reachy.r_arm.elbow.pitch.goto(0)` command is stacked in r_arm gotos queue.
The goto on the elbow will be played when the previous command sent using `goto_joints()` will be over.

The arms should have moved in a way similar to what we saw in the [goto tutorial](2_goto_introduction.ipynb). You already know that you can specify the duration or the interpolation mode of this kind of movement.

We've only seen movements expressed in the 'joint space', i.e., defined by a set of angles. How can we know the position of the gripper in space, or how can we reach an object for which we know its position? That's where kinematics come in.

### Kinematics

The kinematic model describes the motion of a robot in mathematical form without considering the forces and torque affecting it. It only focuses on the geometric relationship between elements.

We have defined the whole kinematic model of the arm. This means the translation and rotation required to go from one joint to the next one. 

[Long story](https://docs.pollen-robotics.com/sdk/first-moves/kinematics/) short, there are two types of kinematics:
- Forward kinematics: from the joints position, the position in space of the gripper is computed
- Inverse kinematics: from a given position of the gripper to reach, all joints positions are computed

> You can easily use forward kinematics and inverse kinematics to switch respectively from joint space to cartesian space and from cartesian space to joint space.

#### Forward Kinematics

Each arm has a `forward_kinematics` method that computes a 4x4 pose matrix (position and orientation of the gripper in space). For instance, the previous movement left the left elbow at 90°. The position of the gripper is:

In [ ]:
reachy.l_arm.forward_kinematics()

It is not mandatory to move the arm to compute forward kinematics. This can be done for any set of joint positions such as:

In [ ]:
reachy.l_arm.forward_kinematics([10, 0, 0, -90, 0, 0, 0])

Reachy didn't move, but you know where it would have gone with an additional 10° applied to the shoulder joint.

#### Inverse Kinematics

Inverse kinematics works in the opposite way. Let's say you want to reach an object for which you know its position. What would be the set of joint positions to provide to `goto_joints`?

In [ ]:
import numpy as np
target = np.identity(4)
target = np.array([[0, 0, -1, 0.3],
    [0, 1, 0, 0.1],
    [1, 0, 0, -0.3],
    [0, 0, 0, 1]])
target

In [ ]:
joints_positions = reachy.l_arm.inverse_kinematics(target)
joints_positions

In [ ]:
reachy.l_arm.goto_joints(joints_positions)

In [12]:
reachy.l_arm.goto_posture("default")
reachy.l_arm.goto_from_matrix(target, wait = True)

> All these goto are illustrated in [draw_square](draw_square.py). Check it out to see how to make Reachy draw a square with its right arm!

### Cartesian space

Controlling the arm in cartesian space allows you to control the position of the gripper in Reachy's coordinate system. It is the recommended way to control the robot for grasping goals.  
Let's got back to the *elbow_90* pose:

In [ ]:
reachy.l_arm.goto_posture('elbow_90', wait = True)

You can easily access the current pose of the gripper using the previously seen method `forward_kinematics()`

In [ ]:
current_pose = reachy.l_arm.forward_kinematics()
print(current_pose)

#### Move the arms in cartesian space

To control the arm in cartesian space, use the `goto_from_matrix(...)` method.  
Let's use it on the current_pose:

In [ ]:
reachy.l_arm.goto_from_matrix(current_pose)

As you have just seen, the arm moved meanwhile the gripper is in the exact same position and orientation: that's because the computed inverse kinematics solution is different from the joints we decided to choose in joint space (`goto_posture()` is in fact a joint space method).  

Let's send the arm to a new goal pose.  
We need to define a pose 4x4 matrix as the new goal pose for the gripper:

In [ ]:
new_pose = current_pose.copy()
new_pose[0, 3] += 0.1
print(new_pose)

This new_pose is translated 10cm front in Reachy's coordinate system (+10cm on Reachy's x axis).  
You can send it to the robot:

In [ ]:
reachy.l_arm.goto_from_matrix(new_pose)

To simplify your life, you have access to functions to easily compute translation or rotation.  
Use the `translate_by(...)` method to send the gripper back to the previous pose, asking for a translation 10cm back (-10cm on Reachy's x axis):

In [ ]:
reachy.l_arm.translate_by(-0.1, 0, 0, frame='robot')

You can easily make the gripper rotate with the `rotate_by(...)` method:

In [ ]:
reachy.l_arm.rotate_by(10, 0, 0, frame='gripper')

The gripper rotate by 10 degrees around the x axis of the gripper.  
For both functions, you need to specify a frame:
* setting **gripper** as frame will translate or rotate in the gripper coordinate system
* setting **robot** as frame will translate or rotate directly in Reachy's coordinate system

If you want to compute a translation or rotation without making the robot move, you can call `get_translation_by(...)` and `get_rotation_by(...)` to get the corresponding pose 4x4 matrix:

In [ ]:
reachy.l_arm.get_translation_by(-0.1, 0.2, -0.1, frame='gripper')

In [ ]:
reachy.l_arm.get_rotation_by(-10, 20, 0, frame='gripper')

## Gripper Control

Finally, you may want to open or close a gripper to grab an object! Use the `close` or `open` method to do so:

In [ ]:
reachy.l_arm.gripper.close()
time.sleep(1)
reachy.l_arm.gripper.open()

Of course, you can partially open the gripper, and get its current state:

In [ ]:
reachy.l_arm.gripper.set_opening(55)
while reachy.l_arm.gripper.is_moving():
    time.sleep(0.1)
reachy.l_arm.gripper.opening

# Set robot back to compliant mode

In [ ]:
reachy.goto_posture('default', duration=5)
reachy.goto_posture('default', wait = True)
    
reachy.turn_off_smoothly()